In [42]:
import numpy as np
import pickle as pkl
import os

from graph_tool import load_graph
from graph_tool.generation import lattice
from graph_tool.spectral import adjacency
from graph_tool import load_graph

from scipy.io import savemat
from glob import glob
from tqdm import tqdm

from cascade_generator import si, observe_cascade
from helpers import infected_nodes, cascade_source, prepare_cascade_for_netfill


In [40]:
graph_name = 'grqc'
infection_proba = 0.1
obs_fraction = 0.5
cascade_fracion = 0.05

In [41]:
g = load_graph('data/{}/graph_weighted_ud_{}.gt'.format(graph_name, infection_proba))

In [43]:
input_dir = 'cascade/{}-msi-s{}-o{}-omuniform/'.format(
    graph_name, cascade_fracion, obs_fraction)
output_dir = 'netfill-cascade/{}-msi-s{}-o{}-omuniform/'.format(
    graph_name, cascade_fracion, obs_fraction)

for input_path in tqdm(glob(input_dir + '*.pkl')):
    output_path = os.path.join(output_dir, os.path.basename(input_path))
    prepare_cascade_for_netfill(g,
                                beta=0.1, p=0.5,
                                input_path=input_path,
                                output_path=output_path)

100%|██████████| 100/100 [00:00<00:00, 417.97it/s]


In [118]:
g.set_directed(True)
edges_iter = list(g.edges())
for e in edges_iter:
    g.add_edge(e.target(), e.source())

In [119]:
edge_weights = g.new_edge_property('float')
edge_weights.set_value(beta)

In [127]:
from sample_pool import TreeSamplePool
from tree_stat import TreeBasedStatistics
from inference import infection_probability
from random_steiner_tree.util import from_gt


gi = from_gt(g, edge_weights)
sampler = TreeSamplePool(g, 1000, 'loop_erased', gi)
sampler.fill(obs)
est = TreeBasedStatistics(g)
# est.build_matrix?
probas = infection_probability(g, obs, sampler=sampler, error_estimator=est)

In [128]:
hidden = list(set(np.arange(g.num_vertices())) - set(obs))

In [1]:
from sklearn.metrics import average_precision_score
average_precision_score(D[hidden], probas[hidden])

NameError: name 'D' is not defined